Dodamo vse importe in definiramo začetno stanje.

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import ast

url_base = "https://urnik.fri.uni-lj.si"

url_year = []
valid_timetables = []

Na url-ju `https://urnik.fri.uni-lj.si/timetable` dobimo seznam vseh urnikov, ki so na voljo. Za nekatere semestre obstaja več urnikov, a ima le en podatke, ostali pa so prazni. Spišemo kodo, ki nam bo poiskala neprazne urnike.

In [ ]:
driver = webdriver.Chrome()
driver.get(url_base + "/timetable")

soup = BeautifulSoup(driver.page_source, 'html.parser')

timetables_section = soup.find('h1', string='Choose timetable').find_parent('div')

# dobimo linke do vseh urnikov
for a_tag in timetables_section.find_all('a'):
    url_year.append(a_tag.get('href'))

# loopamo cez vse urnike
for year in url_year:
    url = url_base + year

    # gremo na zacetno stran urnika za leto in semester v seznamu
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # najdemo sekcijo s skupinami, poiscemo vse programe
    groups_section = soup.find('h2', string='Groups').find_parent('td')
    group_link = groups_section.find_all('a')

    # ce najdemo podatke o skupinah
    if len(group_link) > 0:
        # poiscemo urnik za dodiplomski program vss in preverimo, ali je prazen
        vss = groups_section.find('a', string='1. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')
        vss_url = url_base + vss.get('href')
        
        driver.get(vss_url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ce ima urnik vsaj en vnos/predmet, ga dodamo na seznam
        subject_link = soup.find(class_="link-subject")
        if subject_link:
            valid_timetables.append(vss_url)


# sortiramo in izpisemo
valid_timetables.sort()
index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

Pogledamo sortiran seznam urnikov. Odstranili bomo leto `2018/19` iz analize, saj letni urnik nima podatkov o skupinah. Imamo 4 urnike za leto `2020/21`, ročno odstranimo duplikate, tako kot urnik `test-predavanja`. Odstranimo še duplikat zimskega semestra leta `2023/24`. Ostane nam seznam validnih urnikov med leti `2019/20` in `2024/25`.<br><br>Odstranimo še zadnji del linka, da nam ostane osnovni link do urnika.

In [ ]:
valid_timetables.pop(16)
valid_timetables.pop(13)
valid_timetables.pop(6)
valid_timetables.pop(5)
valid_timetables.pop(0)

valid_timetables = [url.rpartition('/')[0] for url in valid_timetables]

index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

Tako dobljene linke shranimo pretvorimo v DataFrame objekt in jih shranimo v .csv dokument.

In [ ]:
timetables = pd.DataFrame(valid_timetables, columns=['url'])

def extract_year_semester(url):
    years = url.split('fri-')[1].split('-')[0].split('_')
    year = years[0][2:] + "/" + years[1][2:]

    if 'zimski' in url:
        semester = 1
    else:
        semester = 2

    return year, semester

# Apply the function to the DataFrame
timetables[['url_year', 'url_semester']] = timetables['url'].apply(lambda x: pd.Series(extract_year_semester(x)))

timetables.to_csv('timetables.csv', index=False)

print(timetables)

Sedaj se lotimo pridobivanja podatkov o študentih. Najprej po vzorcu zgeneriramo vse vpisne, ki jih bomo preverili.

In [ ]:
timetables = pd.read_csv("./timetables.csv")

# credit to andraz87
student_ids = [
    int(f"63{str(leto_vpisa).zfill(2)}0{str(unique_id).zfill(3)}")
    for leto_vpisa in range(19, 25)
    for unique_id in range(1, 540)
]

# 3234 students to check
# print(len(student_ids))

# transform into a dataframe
students = pd.DataFrame(student_ids, columns=['student_id'])


Najprej sem stestirala kodo na manjšem setu podatkov in hkrati ugotovila, koliko je najvišja individualna vpisna številka.

In [ ]:
# student_ids = [
#     int(f"63{str(leto_vpisa).zfill(2)}0{str(unique_id).zfill(3)}")
#     for leto_vpisa in range(19, 25)
#     for unique_id in range(501, 600)

# rezultat tega je highest_uniq.csv, kjer je najvišja vpisna v formatu 63XX0537
# zaokrožimo in iščemo vpisne od 1 do 540

Spišemo funkcije, ki jih bomo potrebovali:<br>- funkcija `get_url()` za sestavo linka,<br>- funkcija `get_subjects_per_student()`, ki vrne podatke o predmetih, ki jih je imel študent na urniku v danem letu in semestru,<br>- funkcija `get_student_data()`, ki za vsako leto in semester ustvari novo vrstico v dani tabeli in na njej kliče pomožno funkcijo `get_subjects_per_student()`,<br>- in pa še funkcijo `delete_empty_rows()`, ki na koncu izbriše vse študente, ki niso imeli nobenih predmetov čez vsa leta

In [ ]:
# funkcija za sestavo linka
def get_url(year, sem):
    year_str = f"{year}/{year+1}"
    url_row = timetables[(timetables['url_year'] == year_str) & (timetables['url_semester'] == sem)]
    if len(url_row) == 0: return f"ni urlja za leto {year} in semester {sem}"
    url = url_row.iloc[0]['url']
    return url
    

# za danega studenta vrnemo predmete, ki jih je imel na urniku v danem letu in semestru
def get_subjects_per_student(student_id, year, sem, url):
    # preverimo, ali se je student ze sploh vpisal
    if ( int(str(student_id)[2:4]) > int(year[:2]) ):
        return ""
    
    url_student = f"{url}/allocations?student={student_id}"

    try:
        driver = webdriver.Chrome()
        driver.get(url_student)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        subjects_section = soup.find_all(class_='entry-hover')

        subjects_list = []

        # nekateri linki so čudni in imajo preveč predmetov, npr
        # tale "student" ma tko 400 predmetov???
        # url = 'https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-zimski-1-1/allocations?student=63210532'

        #if len(subjects_section) > 15:
        #    print(len(subjects_section))
        #else:

        if len(subjects_section) == 0:
            return ""

        for subject in subjects_section:
            sub = subject.text.split('\n')
            # nekje so tutorske brez idja ki breakajo kodo ce ni vseh teh ifov
            subject_id = ""
            if len(sub) >= 5:
                sub_2 = sub[4].strip()
                if len(sub_2) >= 2:
                    sub_3 = sub_2.split('(')
                    if len(sub_3) >= 2:
                        sub_4 = sub_3[1].split(')')
                        if len(sub_4) >= 1:
                            subject_id = sub_4[0]
                    
            if subject_id not in subjects_list:
                subjects_list.append(subject_id)

    finally:
        driver.quit()

    return subjects_list
    

# ustvarimo novo vrstico v dani tabeli in za vsako celico dobimo podatke o predmetih
def get_student_data(table, year, sem, url):
    year_str = f"{year}/{year+1}"
    col_name = f"{year_str}_{sem}"
    table[col_name] = table['student_id'].apply(lambda x: get_subjects_per_student(x, year_str, sem, url))
    # checkpoint za podatke, ce vmes slucajno kaj crkne
    file_name = f"students_upto_{year}_{sem}.csv"
    table.to_csv(file_name, index=False)


# brisemo vrstice studentov, ki nimajo niti enega predmeta
def delete_empty_rows(data_frame):
    indexes_to_drop = []
    for index, row in data_frame.iterrows():
        if all(isinstance(el, list) and len(el) == 0 or el == "" for el in row[1:]):
            indexes_to_drop.append(index)
    data_frame.drop(indexes_to_drop, inplace=True)

Za vsako kombinacijo leta in semestra poiščemo url, in kličemo funkcijo `get_student_data()`. Končni rezultat shranimo v .csv datoteko.

In [ ]:
for year in range (19, 25):
    for sem in range (1,3):
        url = get_url(year, sem)
        get_student_data(students, year, sem, url)

students.to_csv('students.csv', index=False)

Ustvarimo kopijo podatkov o študentih, izbrišemo odvečne vrstice in rezultat shranimo v .csv datoteko.

In [ ]:
students_no_empty_rows = students.copy()
delete_empty_rows(students_no_empty_rows)

# znebili smo se 3224 - 2763 = 461 študentov
# students_no_empty_rows.count()

students_no_empty_rows.to_csv("./students_no_empty_rows.csv", index=False)